## Learning to Rank

Pointwise: One instance of the set is considered at a time, use any kind of classifier or regressor to predict how relevant it is in the current query. Use each points predicted relevance to order the set.

Pairwise: A pair of instances is chosen and the order of those two is predicted. Repeat this for each pair of the query to find the final order of the entire query.

Listwise: Many or all instances are considered at once. Try to find the optimal order.

- https://www.kaggle.com/code/prashant111/lightgbm-classifier-in-python
- https://www.kaggle.com/code/samratp/lightgbm-xgboost-catboost
- https://developer.nvidia.com/blog/learning-to-rank-with-xgboost-and-gpu/#:~:text=XGBoost%20is%20a%20widely%20used,descent%20using%20an%20objective%20function.
- https://medium.com/predictly-on-tech/learning-to-rank-using-xgboost-83de0166229d

In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt
import seaborn as sns

# Data Preparation (Pre-processing)

In [45]:
import pandas as pd

df = pd.read_csv('https://storage.googleapis.com/data-hotel/list-hotel/Data%20API.csv')
df.head()
#Buat salinan data yang friendly untuk ditampilkan di mobile
df_cp = df.copy()

In [46]:
df2 = pd.read_csv('https://storage.googleapis.com/data-hotel/list-hotel/Data%20API.csv')


In [47]:
df.columns

Index(['id', 'hotel', 'star', 'rating', 'reviews', 'harga', 'places_nearby',
       'facil_+_akomod', 'food_and_drinks_list', 'hotel_services_list',
       'in-room_facilities_list', 'business_facilities_list',
       'nearby_facilities_list', 'public_facilities_list', 'general_list',
       'things_to_do_list', 'accessibilty_list', 'connectivity_list',
       'transportation_list', 'kids_and_pets_list',
       'sports_and_recreations_list', 'shuttle_service_list', 'description'],
      dtype='object')

In [48]:
facilities_output = df2.columns[8:22]
len(facilities_output)
facilities_output

Index(['food_and_drinks_list', 'hotel_services_list',
       'in-room_facilities_list', 'business_facilities_list',
       'nearby_facilities_list', 'public_facilities_list', 'general_list',
       'things_to_do_list', 'accessibilty_list', 'connectivity_list',
       'transportation_list', 'kids_and_pets_list',
       'sports_and_recreations_list', 'shuttle_service_list'],
      dtype='object')

In [53]:
facilities_columns = ['Food and Drinks','Hotel Services','In-room Facilities', 'Business Facilities', 'Nearby Facilities',
                      'Public Facilities', 'General', 'Things to Do', 'Accessibilty', 'Connectivity', 'Transportation',
                      'Kids and Pets', 'Sports and Recreations', 'Shuttle Service']
ind = []
for i in range(len(df)):
    list_subfacil = {}
    facils = df['facil_+_akomod'][i].splitlines()
    facils = [facil for facil in facils if facil != '']
    
        
    for facil in list_subfacil:
        if facil not in facils:
            list_subfacil[facil] = []
    

    for facil in facils:     
        if facil in facilities_columns:
            main_facil = facil
            list_subfacil[main_facil] = []
        else:
            try:
                list_subfacil[main_facil] += [facil]
            except:
                pass
    ind.append(list_subfacil)

len(ind)

89

In [54]:
for index, facil in enumerate(ind):
    for key, value in facil.items():
        key2 = key.lower().split()
        key2 = '_'.join(key2)
        df.at[index, "{}_list".format(key2)] = '\n'.join(value)

In [55]:
df

,id,hotel,star,rating,reviews,harga,places_nearby,facil_+_akomod,food_and_drinks_list,hotel_services_list,...,public_facilities_list,general_list,things_to_do_list,accessibilty_list,connectivity_list,transportation_list,kids_and_pets_list,sports_and_recreations_list,shuttle_service_list,description
0,0,Hotel Indonesia Kempinski Jakarta,5.0,8.9,4363,2.480.500,Nearby Places\n\nJia Jia - Grand Indonesia (De...,Food and Drinks\nA la carte breakfast\nA la ca...,A la carte breakfast\nA la carte dinner\nA la ...,Bellhop\nConcierge\nDoorman\nExpress check-in\...,...,Parking\nCoffee shop\nElevator\n24-hour room s...,AC\nBallroom\nBanquet\nConnecting rooms\nNon-s...,Fitness center\nHealth club\nMassage\nOutdoor ...,Accessible bathroom\nAccessible parking\nRoll-...,INTERNATIONAL_CALL\nFree WiFi\nWiFi (surcharge),Car rental\nValet parking (surcharge),CRIBS\nHighchairs,Fitness center,Airport transfer (surcharge),This Hotel offers ultimate comfort and luxury....
1,1,"The Langham, Jakarta",5.0,8.8,238,3.823.600,Nearby Places\n\nPig Me Up! - Ashta District 8...,Food and Drinks\nA la carte dinner\nA la carte...,A la carte dinner\nA la carte lunch\nBar\nBrea...,Bellhop\nConcierge\nDoorman\nFront desk\n24-ho...,...,Coffee/tea in lobby\nCoffee shop\nElevator\nRe...,AC\nBallroom\nBanquet\nNon-smoking room\nSwimm...,Fitness center\nHealth club\nIndoor pool\nOutd...,NaN,Free WiFi,Secure parking\nValet parking,CRIBS,Fitness center,NaN,This Hotel offers ultimate comfort and luxury....
2,2,Manhattan Hotel,5.0,8.5,8405,6.231.500,Nearby Places\n\nJia Jia - Grand Indonesia (De...,Hotel Services\nBellhop\nConcierge\nMoney chan...,A la carte dinner\nA la carte lunch\nRestauran...,Bellhop\nConcierge\nMoney changer\nDoorman\nEa...,...,Parking\nCoffee/tea in lobby\nCoffee shop\nEar...,AC\nBallroom\nBanquet\nConnecting rooms\nFamil...,Fitness center\nOutdoor pool,Disabled access friendly\nAccessible bathroom\...,INTERNATIONAL_CALL\nLAN Internet (surcharge)\n...,Area shuttle\nParking with surcharge\nSecure p...,Babysitting,Fitness center,Airport transfer (surcharge),This Hotel offers ultimate comfort and luxury....
3,3,Aloft South Jakarta,4.0,8.9,424,762.3,Nearby Places\n\nSouthside Rooftop Bar & Loung...,Public Facilities\nParking\nCoffee shop\nEleva...,Bar\nPoolside bar\nBar,24-hour Receptionist\nLaundry service\nLuggage...,...,Parking\nCoffee shop\nElevator\nRestaurant\nRo...,NaN,Fitness center\nGarden\nOutdoor pool,Disabled access friendly,NaN,Car rental,NaN,Billiard,NaN,This Hotel offers ultimate comfort and luxury....
4,4,"The Mayflower, Jakarta - Marriott Executive Ap...",5.0,9.0,393,1.756.254,Nearby Places\n\nSudirman Plaza\nBusiness\n2 m...,Food and Drinks\nA la carte breakfast\nA la ca...,A la carte breakfast\nA la carte dinner\nA la ...,Welcoming drinks\nConcierge\nMoney changer\nDo...,...,Parking\nCoffee shop\nElevator\n24-hour room s...,AC\nFamily room\nNon-smoking room\nSwimming po...,Fitness center\nHealth club\nHot tub\nIndoor h...,NaN,Free WiFi,Airport transfer\nCar rental\nSecure parking\n...,CRIBS\nHighchairs\nFamily-friendly Facilities\...,Fitness center,Airport transfer (surcharge),This Hotel offers ultimate comfort and luxury....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,84,POP! Hotel Airport Jakarta,2.0,7.9,11026,"385.200,00",Nearby Places\r\n\r\nsTREATs Restaurant - Ibis...,Hotel Services\r\nBellhop\r\n24-hour security\...,NaN,Bellhop\n24-hour security\nLuggage storage\nMu...,...,Parking\nElevator\nSafety deposit box\nWiFi in...,AC\nNon-smoking room\nSmoking area,Garden,Wheelchair accessible,Free WiFi,Airport transfer\nCar rental,NaN,NaN,Airport transfer,This Hotel offers ultimate comfort and luxury....
85,85,Sheraton Grand Jakarta Gandaria City Hotel,5.0,8.9,1511,"2.420.000,00",Nearby Places\r\n\r\nAnigre at Sheraton Gandar...,Food and Drinks\r\nA la carte breakfast\r\nA l...,A la carte breakfast\nA la carte dinner\nA la ...,Bellhop\nWelcoming drinks\nConcierge\nEarly Ch...,...,Parking\nCo

In [ ]:
df_cp.columns

In [ ]:
df_cp.to_csv("Data API.csv")

In [ ]:
contoh = pd.read_csv('Data API.csv')

In [ ]:
contoh.head()

In [58]:
facilities_columns = ['Food and Drinks','Hotel Services','In-room Facilities', 'Business Facilities', 'Nearby Facilities', 'Public Facilities', 'General', 'Things to Do', 'Accessibilty', 'Connectivity', 'Transportation', 'Kids and Pets', 'Sports and Recreations', 'Shuttle Service']

facilities_columns.reverse()

for index , row in df.iterrows():
    
    # split per fasil and akomod
    arr = row['facil_+_akomod'].splitlines() 
    arr = [ar for ar in arr if ar != '']
    #iterate over fasil and akomod

    i = 0
    count = 0

    for  item in reversed(arr):
      count += 1
      if item in facilities_columns:
        df.at[index,item ] = count-1
        count = 0
        i += 1

In [59]:
df.head()

,id,hotel,star,rating,reviews,harga,places_nearby,facil_+_akomod,food_and_drinks_list,hotel_services_list,...,Connectivity,Accessibilty,Things to Do,General,Public Facilities,Nearby Facilities,Business Facilities,In-room Facilities,Hotel Services,Food and Drinks
0,0,Hotel Indonesia Kempinski Jakarta,5.0,8.9,4363,2.480.500,Nearby Places\n\nJia Jia - Grand Indonesia (De...,Food and Drinks\nA la carte breakfast\nA la ca...,A la carte breakfast\nA la carte dinner\nA la ...,Bellhop\nConcierge\nDoorman\nExpress check-in\...,...,3.0,4.0,6.0,7.0,8.0,8.0,8.0,10.0,12.0,24.0
1,1,"The Langham, Jakarta",5.0,8.8,238,3.823.600,Nearby Places\n\nPig Me Up! - Ashta District 8...,Food and Drinks\nA la carte dinner\nA la carte...,A la carte dinner\nA la carte lunch\nBar\nBrea...,Bellhop\nConcierge\nDoorman\nFront desk\n24-ho...,...,1.0,NaN,8.0,5.0,9.0,6.0,5.0,11.0,10.0,12.0
2,2,Manhattan Hotel,5.0,8.5,8405,6.231.500,Nearby Places\n\nJia Jia - Grand Indonesia (De...,Hotel Services\nBellhop\nConcierge\nMoney chan...,A la carte dinner\nA la carte lunch\nRestauran...,Bellhop\nConcierge\nMoney changer\nDoorman\nEa...,...,4.0,6.0,2.0,8.0,14.0,8.0,10.0,11.0,18.0,14.0
3,3,Aloft South Jakarta,4.0,8.9,424,762.3,Nearby Places\n\nSouthside Rooftop Bar & Loung...,Public Facilities\nParking\nCoffee shop\nEleva...,Bar\nPoolside bar\nBar,24-hour Receptionist\nLaundry service\nLuggage...,...,NaN,1.0,3.0,NaN,7.0,NaN,1.0,NaN,3.0,3.0
4,4,"The Mayflower, Jakarta - Marriott Executive Ap...",5.0,9.0,393,1.756.254,Nearby Places\n\nSudirman Plaza\nBusiness\n2 m...,Food and Drinks\nA la carte breakfast\nA la ca...,A la carte breakfast\nA la carte dinner\nA la ...,Welcoming drinks\nConcierge\nMoney changer\nDo...,...,1.0,NaN,10.0,6.0,11.0,4.0,2.0,14.0,13.0,17.0


In [60]:
df = df.fillna(0)

In [62]:
for index , row in df.iterrows():
    
    # split per fasil and akomod
    arr = row['places_nearby'].splitlines() 
    
    #iterate over fasil and akomod

    i = 0
    count = 0
    
    for ind, item in enumerate(arr):
        itemsplits = item.split()
        for x in itemsplits:
            if x.isdigit():
                if itemsplits[1] == "km":
                    meters = itemsplits[0] * 1000
                else:
                    meters = itemsplits[0]
                    
                if meters.isdigit():
                    df.at[index,arr[ind-1]] = meters
                    #print(meters)

In [63]:
print(df.columns)

Index(['id', 'hotel', 'star', 'rating', 'reviews', 'harga', 'places_nearby',
       'facil_+_akomod', 'food_and_drinks_list', 'hotel_services_list',
       'in-room_facilities_list', 'business_facilities_list',
       'nearby_facilities_list', 'public_facilities_list', 'general_list',
       'things_to_do_list', 'accessibilty_list', 'connectivity_list',
       'transportation_list', 'kids_and_pets_list',
       'sports_and_recreations_list', 'shuttle_service_list', 'description',
       'Shuttle Service', 'Sports and Recreations', 'Kids and Pets',
       'Transportation', 'Connectivity', 'Accessibilty', 'Things to Do',
       'General', 'Public Facilities', 'Nearby Facilities',
       'Business Facilities', 'In-room Facilities', 'Hotel Services',
       'Food and Drinks', 'Fast Food', 'Shop & Gifts', 'Business',
       'Transportation Hub', 'Casual Dining', 'Nightlife', 'Park & Zoo',
       'Public Service', 'Arts & Sciences', 'Fine Dining', 'Sport',
       'Quick Bites', 'Education', 

In [65]:
df.harga = df['harga'].str.replace('.','', regex = True)
df.harga = df['harga'].str.replace(',','.', regex = True)
df.harga = df['harga'].astype(float).astype(int)

df.reviews = df['reviews'].str.replace('.','', regex = True)
df.reviews = df['reviews'].str.replace(',','.', regex = True)
df.reviews = df['reviews'].astype(float).astype(int)

In [66]:
df = df.fillna(10000)

In [67]:
c = df.select_dtypes(object).columns
df[c] = df[c].apply(pd.to_numeric,errors='coerce')

In [68]:
df.dtypes

id                               int64
hotel                          float64
star                           float64
rating                         float64
reviews                          int32
harga                            int32
places_nearby                  float64
facil_+_akomod                 float64
food_and_drinks_list           float64
hotel_services_list            float64
in-room_facilities_list        float64
business_facilities_list       float64
nearby_facilities_list         float64
public_facilities_list         float64
general_list                   float64
things_to_do_list              float64
accessibilty_list              float64
connectivity_list              float64
transportation_list            float64
kids_and_pets_list             float64
sports_and_recreations_list    float64
shuttle_service_list           float64
description                    float64
Shuttle Service                float64
Sports and Recreations         float64
Kids and Pets            

In [69]:
df.head()

,id,hotel,star,rating,reviews,harga,places_nearby,facil_+_akomod,food_and_drinks_list,hotel_services_list,...,Fine Dining,Sport,Quick Bites,Education,Street Food,Activity & Games,Cafe,Entertainment,Food Court,Sight & Landmark
0,0,NaN,5.0,8.9,4363,2480500,NaN,NaN,NaN,NaN,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
1,1,NaN,5.0,8.8,238,3823600,NaN,NaN,NaN,NaN,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
2,2,NaN,5.0,8.5,8405,6231500,NaN,NaN,NaN,NaN,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
3,3,NaN,4.0,8.9,424,7623,NaN,NaN,NaN,NaN,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,4,NaN,5.0,9.0,393,1756254,NaN,NaN,NaN,NaN,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000


# Modelling

## Declare feature vector and target variable

In [ ]:
# view summary of dataset
df.info()

## Data Splitting

In [ ]:
X = df[['Star','Reviews','Harga','Shuttle Service','Sports and Recreations', 'Kids and Pets', 'Transportation', 'Connectivity', 'Accessibilty', 'Things to Do', 'General', 'Public Facilities', 'Nearby Facilities', 'Business Facilities', 'In-room Facilities', 'Hotel Services', 'Food and Drinks', 'Fast Food', 'Shop & Gifts', 'Business', 'Transportation Hub', 'Casual Dining', 'Nightlife', 'Park & Zoo', 'Public Service', 'Arts & Sciences', 'Fine Dining', 'Sport', 'Quick Bites', 'Education', 'Street Food', 'Activity & Games', 'Cafe', 'Entertainment', 'Food Court', 'Sight & Landmark' ]]
y = df['Rating']

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
dev_X, val_X, dev_y, val_y = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Light GBM

In [ ]:
#import lightgbm as lgb
#clf = lgb.lambdarank()
#clf.fit(X_train, y_train)

def run_lgb(X_train, X_test, y_train, y_test):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.004,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgtrain = lgb.Dataset(X, label=y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=150, 
                      evals_result=evals_result)
    
    #pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return model, evals_result #pred_test_y

In [ ]:
model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y)
print("LightGBM Training Completed...")

In [ ]:
y_pred = model.predict(X)
print(y_pred)

In [ ]:
'''
# feature importance
print("Features Importance...")
gain = model.feature_importance('Rating')
featureimp = pd.DataFrame({'Rating':model.feature_name(), 
                   'split':model.feature_importance('split'), 
                   'Rating':100 * gain / gain.sum()}).sort_values('Rating', ascending=False)
print(featureimp[:50])
'''

## XGBoost























In [ ]:
def run_xgb(train_X, train_y, val_X, val_y):
    params = {'objective': 'reg:linear', 
          'eval_metric': 'rmse',
          'eta': 0.001,
          'max_depth': 10, 
          'subsample': 0.6, 
          'colsample_bytree': 0.6,
          'alpha':0.001,
          'random_state': 42, 
          'silent': True}
    
    tr_data = xgb.DMatrix(train_X, train_y)
    va_data = xgb.DMatrix(val_X, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 100, verbose_eval=100)
    
    #dtest = xgb.DMatrix(test_X)
    #xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return  model_xgb #, xgb_pred_y,

In [ ]:
# Training XGB
model_xgb = run_xgb(dev_X, dev_y, val_X, val_y)
print("XGB Training Completed...")

## Catboost

In [ ]:
cb_model = CatBoostRegressor(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [ ]:
cb_model.fit(dev_X, dev_y,
             eval_set=(val_X, val_y),
             use_best_model=True,
             verbose=50)

pred_test_cat = np.expm1(cb_model.predict(X))
print(pred_test_cat)

## Tensorflow Regression

In [ ]:
df = df.drop(['Hotel', 'Places Nearby', 'Facil + Akomod'], axis=1)
df

In [ ]:
df2 = df
df3 = df

## Data Preparation

In [ ]:
train_dataset = df.loc[:70]
test_dataset = df.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Rating')
test_labels = test_features.pop('Rating')

In [ ]:
print(train_features.info)

# TENSORFLOW PIPELINE

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

In [ ]:
regularizer = 0.000001
dropout = 0
schedul = -0.0001
lr = 0.001

In [ ]:
def mikro_model(norm):
  
    model = keras.Sequential([
      norm,
      layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(regularizer) ),
      layers.Dropout(dropout),
      layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
      layers.Dropout(dropout),
      layers.Dense(1)
      ])
    
    return model



def small_model(norm):
  
    model = keras.Sequential([
      norm,
      layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(regularizer) ),
        layers.Dropout(dropout),
      layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
      layers.Dense(1)
      ])
    
    return model


def medium_model(norm):
  
    model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(regularizer) ),
      layers.Dropout(dropout),
      layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(regularizer) ),
      layers.Dropout(dropout),
      layers.Dense(1)
      ])
    
    return model


def large_model(norm):
  
    model = keras.Sequential([
      norm,
      layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
      layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
      layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
      layers.Dense(1)
      ])
    
    return model

def scale_model(norm):
  
    model = keras.Sequential([
      norm,
      layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
      layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
      layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
         layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(regularizer)),
        layers.Dropout(dropout),
      layers.Dense(1)
      ])
    
    return model

## OPTIMIZER

In [ ]:
optimizer2 = tf.optimizers.Adam(learning_rate=lr)

## SCHEDULER

In [ ]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(schedul)

In [ ]:
def DNN_Pipeline (model):
    
    model.compile(optimizer= optimizer2, loss='mean_absolute_error')
    
    history = model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler),
    verbose=0, epochs=100)
    
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 2])
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)
    print(hist.tail(10))
    
    return model

In [ ]:
small = DNN_Pipeline(small_model(normalizer))

In [ ]:
medium = DNN_Pipeline(medium_model(normalizer))

In [ ]:
large = DNN_Pipeline(large_model(normalizer))

In [ ]:
mikro = DNN_Pipeline(mikro_model(normalizer))

In [ ]:
scale = DNN_Pipeline(scale_model(normalizer))

In [ ]:
mikro_predict = mikro.predict(test_features)
small_predict = small.predict(test_features)
medium_predict = medium.predict(test_features)
large_predict = large.predict(test_features)
scale_predict = scale.predict(test_features)

In [ ]:
mean_absolute_error(mikro_predict,test_labels)   

In [ ]:
mean_absolute_error(small_predict,test_labels)   

In [ ]:
mean_absolute_error(medium_predict,test_labels)   

In [ ]:
mean_absolute_error(large_predict,test_labels)   

In [ ]:
mean_absolute_error(scale_predict,test_labels)   

In [ ]:
print(scale_predict)

In [ ]:
print (test_labels)

In [ ]:
print(medium.predict(train_features))

# RANGKING WITH REGRESSION

In [ ]:
a = []

for x in range(1,90):
    a.append(x)
    
print(a)

df2 = df
df2['Rank'] = a

df2

In [ ]:
train_dataset = df2.loc[:70]
test_dataset = df2.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Rank')
test_labels = test_features.pop('Rank')

In [ ]:
normalizer2 = tf.keras.layers.Normalization(axis=-1)
normalizer2.adapt(np.array(train_features))
print(normalizer.mean.numpy())

In [ ]:
large = DNN_Pipeline(large_model(normalizer2))


In [ ]:
scale = DNN_Pipeline(scale_model(normalizer2))


In [ ]:
medium = DNN_Pipeline(medium_model(normalizer2))

In [ ]:
df_rank = test_dataset
df_rank['score'] = medium.predict(test_features)
df_rank

In [ ]:
new_sorted = df_rank.sort_values(by=['score'])

# RANk error per item (0 is perfect)

In [ ]:
b = []

for x in range (72,90):
    b.append(x)

    
c = b-(new_sorted['Rank'])

print (c)

In [ ]:
print(df_rank.index)

In [ ]:
test_labels

## Rank Train Datasets

In [ ]:
# Rank train datasets

medium.predict(train_features)

In [ ]:
df_rank = train_dataset
df_rank['score'] = medium.predict(train_features)
df_rank
new_sorted2 = df_rank.sort_values(by=['score'])
new_sorted2

# RANk error per item (0 is perfect)

In [ ]:
K = []

for x in range (1,72):
    K.append(x)

    
c = K-(new_sorted2['Rank'])

for x in c:
    print (x)

## Export the model

In [ ]:
medium.save('TestV1.h5')